In [205]:
# 필요한 라이브러리를 import

# !pip install flatten-dict
import requests
import json
import pandas as pd
import itertools
import time
import math
import numpy as np
from tqdm import tqdm
from flatten_dict import flatten
from flatten_dict.reducer import make_reducer

In [2]:
# riot-API 이용을 위한 api-key값

api_key = 'RGAPI-98a62b70-3c28-4c50-bb65-60eaf49a5cc2'

In [3]:
# 각 tier별 game id를 가져오기위해 url을 저장

IRON_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/IV?page='
IRON_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/III?page='
IRON_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/II?page='
IRON_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/I?page='
BRONZE_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/IV?page='
BRONZE_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/III?page='
BRONZE_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/II?page='
BRONZE_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/I?page='
SILVER_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/IV?page='
SILVER_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/III?page='
SILVER_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/II?page='
SILVER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page='
GOLD_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page='
GOLD_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/III?page='
GOLD_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/II?page='
GOLD_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/I?page='
PLATINUM_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/IV?page='
PLATINUM_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/III?page='
PLATINUM_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/II?page='
PLATINUM_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/I?page='
DIAMOND_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/IV?page='
DIAMOND_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/III?page='
DIAMOND_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/II?page='
DIAMOND_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page='
MASTER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/MASTER/I?page='
GRANDMASTER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page='
CHALLENGER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page='
MATCH_URL = 'https://kr.api.riotgames.com/lol/match/v4/matches/'

In [4]:
# tier의 urel과 api-key를 입력하여 summonerId를 반환받음 return - array

def summonersT(tier_url, api_key):
    print('----------------------------------------------------Summoners start----------------------------------------------------')
    summoners = []
    for i in range(1,5):
        res = requests.get(tier_url+str(i), headers={"X-Riot-Token": api_key})
        summoners.append(json.loads(res.text)[5:105])
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    print('Waiting for summoners end...')
    time.sleep(120)
    print('----------------------------------------------------Summoners done-----------------------------------------------------')
    return pd.DataFrame(list(itertools.chain(*summoners)))['summonerId'].values


#  summoners(iterable)와 api-key를 입력받아 accountId를 반환 return - array

def getAccountId(summoners, api_key):
    print('\n----------------------------------------------------Accounts start-----------------------------------------------------')
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    accountS = []
    time.sleep(2)
    a = 0
    for i  in tqdm(summoners):
        a += 1
        if a % 20 == 0:
            time.sleep(1)
        if a % 100 == 0:
            time.sleep(122)
        togetSummonerId = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/'+str(i)
        res = requests.get(togetSummonerId, headers={"X-Riot-Token": api_key})
        accountS.append(json.loads(res.text))
    print('\n-----------------------------------------------------Accounts done-----------------------------------------------------')   
    return pd.DataFrame(accountS)['accountId'].values


# accountId(iterable)와 api-key를 입력받아 gameId를 반환 return - list

def getMatch(encryptedAccountId, api_key):
    print('\n-----------------------------------------------------MatchId start-----------------------------------------------------')
    a = 0
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    objects = []
    matchIds = []
    time.sleep(2)
    for i in tqdm(encryptedAccountId):
        a += 1
        if a % 20 == 0:
            time.sleep(1)
        if a % 100 == 0:
            time.sleep(122)
        togetMatchOb = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?endIndex=1&beginIndex=0'.format(str(i))
        res = requests.get(togetMatchOb, headers={"X-Riot-Token": api_key})
        ob = pd.DataFrame(json.loads(res.text))
        if 'matches' in list(ob):
            objects.append(pd.DataFrame(ob)['matches'].values[0])
        else:
            print(ob)
            print("----Error recovery time----")
            time.sleep(90)
            rest=getMatchRe([i], api_key)
            matchIds.append(rest)
            print("----Error fixed----")
    matchIds = matchIds + list(pd.DataFrame(objects)['gameId'].values)
    print('\n-----------------------------------------------------MatchId done-----------------------------------------------------')   
    return matchIds


# getMatch중 timeout 오류발생시 다시 request를받기위해 시간간격을 두고 다시 request하도록 함.

def getMatchRe(encryptedAccountId, api_key):
    a = 0
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    objects = []
    matchIds = []
    time.sleep(2)
    for i in tqdm(encryptedAccountId):
        a += 1
        if a % 20 == 0:
            time.sleep(1)
        if a % 100 == 0:
            time.sleep(122)
        togetMatchOb = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?endIndex=1&beginIndex=0'.format(str(i))
        res = requests.get(togetMatchOb, headers={"X-Riot-Token": api_key})
        ob = pd.DataFrame(json.loads(res.text))
        if 'matches' in list(ob):
            objects.append(pd.DataFrame(ob)['matches'].values[0])
        else:
            print(ob)
            print("----Error recovery time----")
            time.sleep(90)
            rest=getMatchRe([i], api_key)
            matchIds.append(rest)
    matchIds = matchIds + list(pd.DataFrame(objects)['gameId'].values)
    return matchIds[0]


# tier와 api-key를 입력하면 gameId를 반환하는 함수

def getMatchId(tier, api_key):
    summoners = summonersT(tier, api_key)
    accountId = getAccountId(summoners, api_key)
    gameId = getMatch(accountId, api_key)
    return gameId

### Get Iron games

In [ ]:
iron_4_gameId = getMatchId(IRON_URL4, api_key)
iron_3_gameId = getMatchId(IRON_URL3, api_key)
iron_2_gameId = getMatchId(IRON_URL2, api_key)
iron_1_gameId = getMatchId(IRON_URL1, api_key)
np.save('gameId/iron/iron_1_gameId', iron_1_gameId)
np.save('gameId/iron/iron_2_gameId', iron_2_gameId)
np.save('gameId/iron/iron_3_gameId', iron_3_gameId)
np.save('gameId/iron/iron_4_gameId', iron_4_gameId)

### Get Bronze games

In [ ]:
bronze_4_gameId = getMatchId(BRONZE_URL4, api_key)
bronze_3_gameId = getMatchId(BRONZE_URL3, api_key)
bronze_2_gameId = getMatchId(BRONZE_URL2, api_key)
bronze_1_gameId = getMatchId(BRONZE_URL1, api_key)
np.save('gameId/bronze/bronze_4_gameId', bronze_4_gameId)
np.save('gameId/bronze/bronze_3_gameId', bronze_3_gameId)
np.save('gameId/bronze/bronze_2_gameId', bronze_2_gameId)
np.save('gameId/bronze/bronze_1_gameId', bronze_1_gameId)

### Get Silver games

In [ ]:
silver_4_gameId = getMatchId(SILVER_URL4, api_key)
silver_3_gameId = getMatchId(SILVER_URL3, api_key)
silver_2_gameId = getMatchId(SILVER_URL2, api_key)
silver_1_gameId = getMatchId(SILVER_URL1, api_key)

np.save('gameId/silver/silver_4_gameId', silver_4_gameId)
np.save('gameId/silver/silver_3_gameId', silver_3_gameId)
np.save('gameId/silver/silver_2_gameId', silver_2_gameId)
np.save('gameId/silver/silver_1_gameId', silver_1_gameId)

### Get Gold games

In [ ]:
gold_4_gameId = getMatchId(GOLD_URL4, api_key)
gold_3_gameId = getMatchId(GOLD_URL3, api_key)
gold_2_gameId = getMatchId(GOLD_URL2, api_key)
gold_1_gameId = getMatchId(GOLD_URL1, api_key)

np.save('gameId/gold/gold_4_gameId', gold_4_gameId)
np.save('gameId/gold/gold_3_gameId', gold_3_gameId)
np.save('gameId/gold/gold_2_gameId', gold_2_gameId)
np.save('gameId/gold/gold_1_gameId', gold_1_gameId)

### Get platinum games

In [ ]:
platinum_4_gameId = getMatchId(PLATINUM_URL4, api_key)
platinum_3_gameId = getMatchId(PLATINUM_URL3, api_key)
platinum_2_gameId = getMatchId(PLATINUM_URL2, api_key)
platinum_1_gameId = getMatchId(PLATINUM_URL1, api_key)

np.save('gameId/platinum/platinum_4_gameId', platinum_4_gameId)
np.save('gameId/platinum/platinum_3_gameId', platinum_3_gameId)
np.save('gameId/platinum/platinum_2_gameId', platinum_2_gameId)
np.save('gameId/platinum/platinum_1_gameId', platinum_1_gameId)

### Get diamond, Master, GrandMaster, challenger games

In [ ]:
diamond_4_gameId = getMatchId(DIAMOND_URL4, api_key)
diamond_3_gameId = getMatchId(DIAMOND_URL3, api_key)
diamond_2_gameId = getMatchId(DIAMOND_URL2, api_key)
diamond_1_gameId = getMatchId(DIAMOND_URL1, api_key)
master_gameId = getMatchId(MASTER_URL1, api_key)
grandmaster_gameId = getMatchId(GRANDMASTER_URL1, api_key)
challenger_gameId = getMatchId(CHALLENGER_URL1, api_key)

np.save('gameId/diamond/diamond_4_gameId', diamond_4_gameId)
np.save('gameId/diamond/diamond_3_gameId', diamond_3_gameId)
np.save('gameId/diamond/diamond_2_gameId', diamond_2_gameId)
np.save('gameId/diamond/diamond_1_gameId', diamond_1_gameId)
np.save('gameId/master/master_gameId', master_gameId)
np.save('gameId/grandmaster/grandmaster_gameId', grandmaster_gameId)
np.save('gameId/challenger/challenger_gameId', challenger_gameId)

# Testing

In [5]:
a = np.load('gameId/diamond/diamond_1_gameId.npy')
len(set(a.tolist()))

353

In [ ]:
a = np.load('gameId/diamond/diamond_1_gameId.npy')
print(a)

In [7]:
matches=[]
for i in tqdm(a[:90]):
    load_url = MATCH_URL + str(i)
    res = requests.get(load_url, headers={"X-Riot-Token": api_key})
    matches.append(json.loads(res.text))

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:33<00:00,  2.70it/s]


In [ ]:
matches[0]
del(matches[0][])

In [ ]:
temp = (flatten(matches[1], enumerate_types=(list,), reducer=make_reducer(delimiter='_')))
for k,v in temp.items():
    print(k, v)

In [355]:
def flatter(matches):
    matchlist = []
    for i in matches:
        matchlist.append(flatten(i, enumerate_types=(list,), reducer=make_reducer(delimiter='_')))
    
    return matchlist


def filter_dict(d, f):
    ''' Filters dictionary d by function f. '''
    newDict = dict()
    # Iterate over all (k,v) pairs in names
    for key, value in d.items():
        # Is condition satisfied?
        if f(key, value):
            newDict[key] = value
    return newDict



def filter_team_dict(d, f):
    ''' Filters dictionary d by function f. '''
    newDict = dict()
    # Iterate over all (k,v) pairs in names
    ind = -1
    kl = ['teams_win', 'teams_firstBlood', 'teams_firstTower', 'teams_firstInhibitor', 'teams_firstBaron', 'teams_firstDragon', 'teams_firstRiftHerald', 'teams_towerKills', 'teams_inhibitorKills', 'teams_baronKills', 'teams_dragonKills', 'teams_riftHeraldKills']
    for key, value in d.items():
        # Is condition satisfied?
        if f(key, value):
            ind += 1
            newDict[kl[ind]] = value
    return newDict



def filter_participant0_dict(d, f):
    ''' Filters dictionary d by function f. '''
    newDict = dict()
    # Iterate over all (k,v) pairs in names
    for key, value in d.items():
        # Is condition satisfied?
        if f(key, value):
            temp = int(key[13])
            pkey = str(temp)+key[14:]
            newDict[pkey] = value
    return newDict

def filter_participant5_dict(d, f):
    ''' Filters dictionary d by function f. '''
    newDict = dict()
    # Iterate over all (k,v) pairs in names
    for key, value in d.items():
        # Is condition satisfied?
        if f(key, value):
            temp = int(key[13])
            pkey = str(temp-5)+key[14:]
            newDict[pkey] = value
    return newDict

def winTeams_0(match):
    matchtemp = match
    
    for delstring in ['item', 'Id', 'playerScore', 'perk', 'Perk', 'bans', 'PlayerScore', 'timeline_csDiffPerMinDeltas_0-10', 'stats_totalScoreRank', 'dominionVictoryScore', 'stats_win', 'stats_unrealKills', 'stats_sightWardsBoughtInGame', 'timeline', 'vilemawKills', 'dominionVictoryScore','stats_firstInhibitorAssist','stats_firstInhibitorKill', 'stats_firstTowerAssist', 'stats_firstTowerKill', 'stats_firstBloodAssist', 'stats_firstBloodKill']: 
        temp = list(filter_dict(match, lambda k,v: delstring in k).keys())
        for i in temp:
            del(matchtemp[i])
    win.update(filter_participant0_dict(matchtemp, lambda k,v: 'participants_0' in k))
    print(win)
    win.update(filter_participant0_dict(matchtemp, lambda k,v: 'participants_1' in k))
    win.update(filter_participant0_dict(matchtemp, lambda k,v: 'participants_2' in k))
    win.update(filter_participant0_dict(matchtemp, lambda k,v: 'participants_3' in k))
    win.update(filter_participant0_dict(matchtemp, lambda k,v: 'participants_4' in k))

    loss.update(filter_participant5_dict(matchtemp, lambda k,v: 'participants_5' in k))
    loss.update(filter_participant5_dict(matchtemp, lambda k,v: 'participants_6' in k))
    loss.update(filter_participant5_dict(matchtemp, lambda k,v: 'participants_7' in k))
    loss.update(filter_participant5_dict(matchtemp, lambda k,v: 'participants_8' in k))
    loss.update(filter_participant5_dict(matchtemp, lambda k,v: 'participants_9' in k))
    
    win = filter_team_dict(matchtemp, lambda k,v: 'teams_0' in k)
    loss = filter_team_dict(matchtemp, lambda k,v: 'teams_1' in k)
    
    sdfsfdsdfsdfsfd
    
    return [win, loss]


def winTeams_1(match):
    temp = winTeams_0(match)
    return [temp[1], temp[0]]


def divideTeams(matches):
    resultList = list()
    for i in matches:
        lossDict = dict()
        winDict = dict()
        lossDict.update({'gameDuration':i['gameDuration']})
        winDict.update({'gameDuration':i['gameDuration']})
        lossDict.update({'gameMode':i['gameMode']})
        winDict.update({'gameMode':i['gameMode']})
        
        if i['teams_0_win'] == 'Win':
            winDict.update(winTeams_0(i)[0])
            lossDict.update(winTeams_0(i)[1])
        else:
            winDict.update(winTeams_1(i)[0])
            lossDict.update(winTeams_1(i)[1])
        resultList.append(winDict)
        resultList.append(lossDict)
    return resultList   

# def avgPlayer(dataframe):
#     for i in range(5):
        

In [356]:
temp1 = filter_dict(temp, lambda k,v: 'item' in k)


In [ ]:
a = divideTeams(flatter(matches))
# for k,v in flatter(matches)[0].items():
#     print(k , v)

In [360]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None


df = pd.DataFrame(a)
df["teams_firstBlood"] = df["teams_firstBlood"].astype(int)
df["teams_firstTower"] = df["teams_firstTower"].astype(int)
df["teams_firstInhibitor"] = df["teams_firstInhibitor"].astype(int)
df["teams_firstBaron"] = df["teams_firstBaron"].astype(int)
df["teams_firstDragon"] = df["teams_firstDragon"].astype(int)
df["teams_firstRiftHerald"] = df["teams_firstRiftHerald"].astype(int)
df = df.loc[df['gameMode'] == 'CLASSIC']

    

df


,gameDuration,gameMode,teams_win,teams_firstBlood,teams_firstTower,teams_firstInhibitor,teams_firstBaron,teams_firstDragon,teams_firstRiftHerald,teams_towerKills,teams_inhibitorKills,teams_baronKills,teams_dragonKills,teams_riftHeraldKills,0_stats_kills,0_stats_deaths,0_stats_assists,0_stats_largestKillingSpree,0_stats_largestMultiKill,0_stats_killingSprees,0_stats_longestTimeSpentLiving,0_stats_doubleKills,0_stats_tripleKills,0_stats_quadraKills,0_stats_pentaKills,0_stats_totalDamageDealt,0_stats_magicDamageDealt,0_stats_physicalDamageDealt,0_stats_trueDamageDealt,0_stats_largestCriticalStrike,0_stats_totalDamageDealtToChampions,0_stats_magicDamageDealtToChampions,0_stats_physicalDamageDealtToChampions,0_stats_trueDamageDealtToChampions,0_stats_totalHeal,0_stats_totalUnitsHealed,0_stats_damageSelfMitigated,0_stats_damageDealtToObjectives,0_stats_damageDealtToTurrets,0_stats_visionScore,0_stats_timeCCingOthers,0_stats_totalDamageTaken,0_stats_magicalDamageTaken,0_stats_physicalDamageTaken,0_stats_trueDamageTaken,0_stats_goldEarned,0_stats_goldSpent,0_stats_turretKills,0_stats_inhibitorKills,0_stats_totalMinionsKilled,0_stats_neutralMinionsKilled,0_stats_totalTimeCrowdControlDealt,0_stats_champLevel,0_stats_visionWardsBoughtInGame,1_stats_kills,1_stats_deaths,1_stats_assists,1_stats_largestKillingSpree,1_stats_largestMultiKill,1_stats_killingSprees,1_stats_longestTimeSpentLiving,1_stats_doubleKills,1_stats_tripleKills,1_stats_quadraKills,1_stats_pentaKills,1_stats_totalDamageDealt,1_stats_magicDamageDealt,1_stats_physicalDamageDealt,1_stats_trueDamageDealt,1_stats_largestCriticalStrike,1_stats_totalDamageDealtToChampions,1_stats_magicDamageDealtToChampions,1_stats_physicalDamageDealtToChampions,1_stats_trueDamageDealtToChampions,1_stats_totalHeal,1_stats_totalUnitsHealed,1_stats_damageSelfMitigated,1_stats_damageDealtToObjectives,1_stats_damageDealtToTurrets,1_stats_visionScore,1_stats_timeCCingOthers,1_stats_totalDamageTaken,1_stats_magicalDamageTaken,1_stats_physicalDamageTaken,1_stats_trueDamageTaken,1_stats_goldEarned,1_stats_goldSpent,1_stats_turretKills,1_stats_inhibitorKills,1_stats_totalMinionsKilled,1_stats_neutralMinionsKilled,1_stats_totalTimeCrowdControlDealt,1_stats_champLevel,1_stats_visionWardsBoughtInGame,2_stats_kills,2_stats_deaths,2_stats_assists,2_stats_largestKillingSpree,2_stats_largestMultiKill,2_stats_killingSprees,2_stats_longestTimeSpentLiving,2_stats_doubleKills,2_stats_tripleKills,2_stats_quadraKills,2_stats_pentaKills,2_stats_totalDamageDealt,2_stats_magicDamageDealt,2_stats_physicalDamageDealt,2_stats_trueDamageDealt,2_stats_largestCriticalStrike,2_stats_totalDamageDealtToChampions,2_stats_magicDamageDealtToChampions,2_stats_physicalDamageDealtToChampions,2_stats_trueDamageDealtToChampions,2_stats_totalHeal,2_stats_totalUnitsHealed,2_stats_damageSelfMitigated,2_stats_damageDealtToObjectives,2_stats_damageDealtToTurrets,2_stats_visionScore,2_stats_timeCCingOthers,2_stats_totalDamageTaken,2_stats_magicalDamageTaken,2_stats_physicalDamageTaken,2_stats_trueDamageTaken,2_stats_goldEarned,2_stats_goldSpent,2_stats_turretKills,2_stats_inhibitorKills,2_stats_totalMinionsKilled,2_stats_neutralMinionsKilled,2_stats_totalTimeCrowdControlDealt,2_stats_champLevel,2_stats_visionWardsBoughtInGame,3_stats_kills,3_stats_deaths,3_stats_assists,3_stats_largestKillingSpree,3_stats_largestMultiKill,3_stats_killingSprees,3_stats_longestTimeSpentLiving,3_stats_doubleKills,3_stats_tripleKills,3_stats_quadraKills,3_stats_pentaKills,3_stats_totalDamageDealt,3_stats_magicDamageDealt,3_stats_physicalDamageDealt,3_stats_trueDamageDealt,3_stats_largestCriticalStrike,3_stats_totalDamageDealtToChampions,3_stats_magicDamageDealtToChampions,3_stats_physicalDamageDealtToChampions,3_stats_trueDamageDealtToChampions,3_stats_totalHeal,3_stats_totalUnitsHealed,3_stats_damageSelfMitigated,3_stats_damageDealtToObjectives,3_stats_damageDealtToTurrets,3_stats_visionScore,3_stats_timeCCingOthers,3_stats_totalDamageTaken,3_stats_magicalDamage

In [53]:
for k in temp1.keys():
    print(k)

participants_0_stats_item0
participants_0_stats_item1
participants_0_stats_item2
participants_0_stats_item3
participants_0_stats_item4
participants_0_stats_item5
participants_0_stats_item6
participants_1_stats_item0
participants_1_stats_item1
participants_1_stats_item2
participants_1_stats_item3
participants_1_stats_item4
participants_1_stats_item5
participants_1_stats_item6
participants_2_stats_item0
participants_2_stats_item1
participants_2_stats_item2
participants_2_stats_item3
participants_2_stats_item4
participants_2_stats_item5
participants_2_stats_item6
participants_3_stats_item0
participants_3_stats_item1
participants_3_stats_item2
participants_3_stats_item3
participants_3_stats_item4
participants_3_stats_item5
participants_3_stats_item6
participants_4_stats_item0
participants_4_stats_item1
participants_4_stats_item2
participants_4_stats_item3
participants_4_stats_item4
participants_4_stats_item5
participants_4_stats_item6
participants_5_stats_item0
participants_5_stats_item1
p

In [106]:
gameData = []
for i in matches:
    gameData.append(flatten(i, enumerate_types=(list,), reducer=make_reducer(delimiter='_')))

In [354]:
a = {'asd' : 10, 'sdf' : 20}
b = {'asd' : 100, 'sdf' : 220}
c = np.array(list(b.values())) + np.array(list(a.values()))
c

array([110, 240])

In [128]:

tes = pd.DataFrame(gameData)
aaaaaa = tes.keys()
for i in aaaaaa:
    print(i)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
def divTeam(gamedata):
gameDuration
teams_0_teamId
teams_0_win
teams_0_firstBlood
teams_0_firstTower
teams_0_firstInhibitor
teams_0_firstBaron
teams_0_firstDragon
teams_0_firstRiftHerald
teams_0_towerKills
teams_0_inhibitorKills
teams_0_baronKills
teams_0_dragonKills
teams_0_riftHeraldKills



participants_0_participantId
participants_0_teamId
participants_0_championId
participants_0_spell1Id
participants_0_spell2Id
participants_0_stats_participantId
participants_0_stats_win
participants_0_stats_item0
participants_0_stats_item1
participants_0_stats_item2
participants_0_stats_item3
participants_0_stats_item4
participants_0_stats_item5
participants_0_stats_item6
participants_0_stats_kills
participants_0_stats_deaths
participants_0_stats_assists
participants_0_stats_largestKillingSpree
participants_0_stats_largestMultiKill
participants_0_stats_killingSprees
participants_0_stats_longestTimeSpentLiving
participants_0_stats_doubleKills
participants_0_stats_tripleKills
participants_0_stats_quadraKills
participants_0_stats_pentaKills
participants_0_stats_unrealKills
participants_0_stats_totalDamageDealt
participants_0_stats_magicDamageDealt
participants_0_stats_physicalDamageDealt
participants_0_stats_trueDamageDealt
participants_0_stats_largestCriticalStrike
participants_0_stats_totalDamageDealtToChampions
participants_0_stats_magicDamageDealtToChampions
participants_0_stats_physicalDamageDealtToChampions
participants_0_stats_trueDamageDealtToChampions
participants_0_stats_totalHeal
participants_0_stats_totalUnitsHealed
participants_0_stats_damageSelfMitigated
participants_0_stats_damageDealtToObjectives
participants_0_stats_damageDealtToTurrets
participants_0_stats_visionScore
participants_0_stats_timeCCingOthers
participants_0_stats_totalDamageTaken
participants_0_stats_magicalDamageTaken
participants_0_stats_physicalDamageTaken
participants_0_stats_trueDamageTaken
participants_0_stats_goldEarned
participants_0_stats_goldSpent
participants_0_stats_turretKills
participants_0_stats_inhibitorKills
participants_0_stats_totalMinionsKilled
participants_0_stats_neutralMinionsKilled
participants_0_stats_totalTimeCrowdControlDealt
participants_0_stats_champLevel
participants_0_stats_visionWardsBoughtInGame
participants_0_stats_sightWardsBoughtInGame
participants_0_stats_firstBloodKill
participants_0_stats_firstBloodAssist
participants_0_stats_firstTowerKill
participants_0_stats_firstTowerAssist
participants_0_stats_firstInhibitorKill
participants_0_stats_firstInhibitorAssist
participants_0_stats_combatPlayerScore
participants_0_stats_objectivePlayerScore
participants_0_stats_totalPlayerScore
participants_0_stats_totalScoreRank
participants_0_stats_playerScore0
participants_0_stats_playerScore1
participants_0_stats_playerScore2
participants_0_stats_playerScore3
participants_0_stats_playerScore4
participants_0_stats_playerScore5
participants_0_stats_playerScore6
participants_0_stats_playerScore7
participants_0_stats_playerScore8
participants_0_stats_playerScore9
participants_0_stats_perk0
participants_0_stats_perk0Var1
participants_0_stats_perk0Var2
participants_0_stats_perk0Var3
participants_0_stats_perk1
participants_0_stats_perk1Var1
participants_0_stats_perk1Var2
participants_0_stats_perk1Var3
participants_0_stats_perk2
participants_0_stats_perk2Var1
participants_0_stats_perk2Var2
participants_0_stats_perk2Var3
participants_0_stats_perk3
participants_0_stats_perk3Var1
participants_0_stats_perk3Var2
participants_0_stats_perk3Var3
participants_0_stats_perk4
participants_0_stats_perk4Var1
participants_0_stats_perk4Var2
participants_0_stats_perk4Var3
participants_0_stats_perk5
participants_0_stats_perk5Var1
participants_0_stats_perk5Var2
participants_0_stats_perk5Var3
participants_0_stats_perkPrimaryStyle
participants_0_stats_perkSubStyle
participants_0_stats_statPerk0
participants_0_stats_statPerk1
participants_0_stats_statPerk2
participants_0_timeline_participantId
participants_0_timeline_creepsPerMinDeltas_0-10
participants_0_timeline_xpPerMinDeltas_0-10
participants_0_timeline_goldPerMinDeltas_0-10
participants_0_timeline_csDiffPerMinDeltas_0-10
participants_0_timeline_xpDiffPerMinDeltas_0-10
participants_0_timeline_damageTakenPerMinDeltas_0-10
participants_0_timeline_damageTakenDiffPerMinDeltas_0-10
participants_0_timeline_role
participants_0_timeline_lane
participants_1_participantId
participants_1_teamId
participants_1_championId
participants_1_spell1Id
participants_1_spell2Id
participants_1_stats_participantId
participants_1_stats_win
participants_1_stats_item0
participants_1_stats_item1
participants_1_stats_item2
participants_1_stats_item3
participants_1_stats_item4
participants_1_stats_item5
participants_1_stats_item6
participants_1_stats_kills
participants_1_stats_deaths
participants_1_stats_assists
participants_1_stats_largestKillingSpree
participants_1_stats_largestMultiKill
participants_1_stats_killingSprees
participants_1_stats_longestTimeSpentLiving
participants_1_stats_doubleKills
participants_1_stats_tripleKills
participants_1_stats_quadraKills
participants_1_stats_pentaKills
participants_1_stats_unrealKills
participants_1_stats_totalDamageDealt
participants_1_stats_magicDamageDealt
participants_1_stats_physicalDamageDealt
participants_1_stats_trueDamageDealt
participants_1_stats_largestCriticalStrike
participants_1_stats_totalDamageDealtToChampions
participants_1_stats_magicDamageDealtToChampions
participants_1_stats_physicalDamageDealtToChampions
participants_1_stats_trueDamageDealtToChampions
participants_1_stats_totalHeal
participants_1_stats_totalUnitsHealed
participants_1_stats_damageSelfMitigated
participants_1_stats_damageDealtToObjectives
participants_1_stats_damageDealtToTurrets
participants_1_stats_visionScore
participants_1_stats_timeCCingOthers
participants_1_stats_totalDamageTaken
participants_1_stats_magicalDamageTaken
participants_1_stats_physicalDamageTaken
participants_1_stats_trueDamageTaken
participants_1_stats_goldEarned
participants_1_stats_goldSpent
participants_1_stats_turretKills
participants_1_stats_inhibitorKills
participants_1_stats_totalMinionsKilled
participants_1_stats_neutralMinionsKilled
participants_1_stats_totalTimeCrowdControlDealt
participants_1_stats_champLevel
participants_1_stats_visionWardsBoughtInGame
participants_1_stats_sightWardsBoughtInGame
participants_1_stats_firstBloodKill
participants_1_stats_firstBloodAssist
participants_1_stats_firstTowerKill
participants_1_stats_firstTowerAssist
participants_1_stats_firstInhibitorKill
participants_1_stats_firstInhibitorAssist
participants_1_stats_combatPlayerScore
participants_1_stats_objectivePlayerScore
participants_1_stats_totalPlayerScore
participants_1_stats_totalScoreRank
participants_1_stats_playerScore0
participants_1_stats_playerScore1
participants_1_stats_playerScore2
participants_1_stats_playerScore3
participants_1_stats_playerScore4
participants_1_stats_playerScore5
participants_1_stats_playerScore6
participants_1_stats_playerScore7
participants_1_stats_playerScore8
participants_1_stats_playerScore9
participants_1_stats_perk0
participants_1_stats_perk0Var1
participants_1_stats_perk0Var2
participants_1_stats_perk0Var3
participants_1_stats_perk1
participants_1_stats_perk1Var1
participants_1_stats_perk1Var2
participants_1_stats_perk1Var3
participants_1_stats_perk2
participants_1_stats_perk2Var1
participants_1_stats_perk2Var2
participants_1_stats_perk2Var3
participants_1_stats_perk3
participants_1_stats_perk3Var1
participants_1_stats_perk3Var2
participants_1_stats_perk3Var3
participants_1_stats_perk4
participants_1_stats_perk4Var1
participants_1_stats_perk4Var2
participants_1_stats_perk4Var3
participants_1_stats_perk5
participants_1_stats_perk5Var1
participants_1_stats_perk5Var2
participants_1_stats_perk5Var3
participants_1_stats_perkPrimaryStyle
participants_1_stats_perkSubStyle
participants_1_stats_statPerk0
participants_1_stats_statPerk1
participants_1_stats_statPerk2
participants_1_timeline_participantId
participants_1_timeline_creepsPerMinDeltas_0-10
participants_1_timeline_xpPerMinDeltas_0-10
participants_1_timeline_goldPerMinDeltas_0-10
participants_1_timeline_csDiffPerMinDeltas_0-10
participants_1_timeline_xpDiffPerMinDeltas_0-10
participants_1_timeline_damageTakenPerMinDeltas_0-10
participants_1_timeline_damageTakenDiffPerMinDeltas_0-10
participants_1_timeline_role
participants_1_timeline_lane
participants_2_participantId
participants_2_teamId
participants_2_championId
participants_2_spell1Id
participants_2_spell2Id
participants_2_stats_participantId
participants_2_stats_win
participants_2_stats_item0
participants_2_stats_item1
participants_2_stats_item2
participants_2_stats_item3
participants_2_stats_item4
participants_2_stats_item5
participants_2_stats_item6
participants_2_stats_kills
participants_2_stats_deaths
participants_2_stats_assists
participants_2_stats_largestKillingSpree
participants_2_stats_largestMultiKill
participants_2_stats_killingSprees
participants_2_stats_longestTimeSpentLiving
participants_2_stats_doubleKills
participants_2_stats_tripleKills
participants_2_stats_quadraKills
participants_2_stats_pentaKills
participants_2_stats_unrealKills
participants_2_stats_totalDamageDealt
participants_2_stats_magicDamageDealt
participants_2_stats_physicalDamageDealt
participants_2_stats_trueDamageDealt
participants_2_stats_largestCriticalStrike
participants_2_stats_totalDamageDealtToChampions
participants_2_stats_magicDamageDealtToChampions
participants_2_stats_physicalDamageDealtToChampions
participants_2_stats_trueDamageDealtToChampions
participants_2_stats_totalHeal
participants_2_stats_totalUnitsHealed
participants_2_stats_damageSelfMitigated
participants_2_stats_damageDealtToObjectives
participants_2_stats_damageDealtToTurrets
participants_2_stats_visionScore
participants_2_stats_timeCCingOthers
participants_2_stats_totalDamageTaken
participants_2_stats_magicalDamageTaken
participants_2_stats_physicalDamageTaken
participants_2_stats_trueDamageTaken
participants_2_stats_goldEarned
participants_2_stats_goldSpent
participants_2_stats_turretKills
participants_2_stats_inhibitorKills
participants_2_stats_totalMinionsKilled
participants_2_stats_neutralMinionsKilled
participants_2_stats_totalTimeCrowdControlDealt
participants_2_stats_champLevel
participants_2_stats_visionWardsBoughtInGame
participants_2_stats_sightWardsBoughtInGame
participants_2_stats_firstBloodKill
participants_2_stats_firstBloodAssist
participants_2_stats_firstTowerKill
participants_2_stats_firstTowerAssist
participants_2_stats_firstInhibitorKill
participants_2_stats_firstInhibitorAssist
participants_2_stats_combatPlayerScore
participants_2_stats_objectivePlayerScore
participants_2_stats_totalPlayerScore
participants_2_stats_totalScoreRank
participants_2_stats_playerScore0
participants_2_stats_playerScore1
participants_2_stats_playerScore2
participants_2_stats_playerScore3
participants_2_stats_playerScore4
participants_2_stats_playerScore5
participants_2_stats_playerScore6
participants_2_stats_playerScore7
participants_2_stats_playerScore8
participants_2_stats_playerScore9
participants_2_stats_perk0
participants_2_stats_perk0Var1
participants_2_stats_perk0Var2
participants_2_stats_perk0Var3
participants_2_stats_perk1
participants_2_stats_perk1Var1
participants_2_stats_perk1Var2
participants_2_stats_perk1Var3
participants_2_stats_perk2
participants_2_stats_perk2Var1
participants_2_stats_perk2Var2
participants_2_stats_perk2Var3
participants_2_stats_perk3
participants_2_stats_perk3Var1
participants_2_stats_perk3Var2
participants_2_stats_perk3Var3
participants_2_stats_perk4
participants_2_stats_perk4Var1
participants_2_stats_perk4Var2
participants_2_stats_perk4Var3
participants_2_stats_perk5
participants_2_stats_perk5Var1
participants_2_stats_perk5Var2
participants_2_stats_perk5Var3
participants_2_stats_perkPrimaryStyle
participants_2_stats_perkSubStyle
participants_2_stats_statPerk0
participants_2_stats_statPerk1
participants_2_stats_statPerk2
participants_2_timeline_participantId
participants_2_timeline_creepsPerMinDeltas_0-10
participants_2_timeline_xpPerMinDeltas_0-10
participants_2_timeline_goldPerMinDeltas_0-10
participants_2_timeline_csDiffPerMinDeltas_0-10
participants_2_timeline_xpDiffPerMinDeltas_0-10
participants_2_timeline_damageTakenPerMinDeltas_0-10
participants_2_timeline_damageTakenDiffPerMinDeltas_0-10
participants_2_timeline_role
participants_2_timeline_lane
participants_3_participantId
participants_3_teamId
participants_3_championId
participants_3_spell1Id
participants_3_spell2Id
participants_3_stats_participantId
participants_3_stats_win
participants_3_stats_item0
participants_3_stats_item1
participants_3_stats_item2
participants_3_stats_item3
participants_3_stats_item4
participants_3_stats_item5
participants_3_stats_item6
participants_3_stats_kills
participants_3_stats_deaths
participants_3_stats_assists
participants_3_stats_largestKillingSpree
participants_3_stats_largestMultiKill
participants_3_stats_killingSprees
participants_3_stats_longestTimeSpentLiving
participants_3_stats_doubleKills
participants_3_stats_tripleKills
participants_3_stats_quadraKills
participants_3_stats_pentaKills
participants_3_stats_unrealKills
participants_3_stats_totalDamageDealt
participants_3_stats_magicDamageDealt
participants_3_stats_physicalDamageDealt
participants_3_stats_trueDamageDealt
participants_3_stats_largestCriticalStrike
participants_3_stats_totalDamageDealtToChampions
participants_3_stats_magicDamageDealtToChampions
participants_3_stats_physicalDamageDealtToChampions
participants_3_stats_trueDamageDealtToChampions
participants_3_stats_totalHeal
participants_3_stats_totalUnitsHealed
participants_3_stats_damageSelfMitigated
participants_3_stats_damageDealtToObjectives
participants_3_stats_damageDealtToTurrets
participants_3_stats_visionScore
participants_3_stats_timeCCingOthers
participants_3_stats_totalDamageTaken
participants_3_stats_magicalDamageTaken
participants_3_stats_physicalDamageTaken
participants_3_stats_trueDamageTaken
participants_3_stats_goldEarned
participants_3_stats_goldSpent
participants_3_stats_turretKills
participants_3_stats_inhibitorKills
participants_3_stats_totalMinionsKilled
participants_3_stats_neutralMinionsKilled
participants_3_stats_totalTimeCrowdControlDealt
participants_3_stats_champLevel
participants_3_stats_visionWardsBoughtInGame
participants_3_stats_sightWardsBoughtInGame
participants_3_stats_firstBloodKill
participants_3_stats_firstBloodAssist
participants_3_stats_firstTowerKill
participants_3_stats_firstTowerAssist
participants_3_stats_firstInhibitorKill
participants_3_stats_firstInhibitorAssist
participants_3_stats_combatPlayerScore
participants_3_stats_objectivePlayerScore
participants_3_stats_totalPlayerScore
participants_3_stats_totalScoreRank
participants_3_stats_playerScore0
participants_3_stats_playerScore1
participants_3_stats_playerScore2
participants_3_stats_playerScore3
participants_3_stats_playerScore4
participants_3_stats_playerScore5
participants_3_stats_playerScore6
participants_3_stats_playerScore7
participants_3_stats_playerScore8
participants_3_stats_playerScore9
participants_3_stats_perk0
participants_3_stats_perk0Var1
participants_3_stats_perk0Var2
participants_3_stats_perk0Var3
participants_3_stats_perk1
participants_3_stats_perk1Var1
participants_3_stats_perk1Var2
participants_3_stats_perk1Var3
participants_3_stats_perk2
participants_3_stats_perk2Var1
participants_3_stats_perk2Var2
participants_3_stats_perk2Var3
participants_3_stats_perk3
participants_3_stats_perk3Var1
participants_3_stats_perk3Var2
participants_3_stats_perk3Var3
participants_3_stats_perk4
participants_3_stats_perk4Var1
participants_3_stats_perk4Var2
participants_3_stats_perk4Var3
participants_3_stats_perk5
participants_3_stats_perk5Var1
participants_3_stats_perk5Var2
participants_3_stats_perk5Var3
participants_3_stats_perkPrimaryStyle
participants_3_stats_perkSubStyle
participants_3_stats_statPerk0
participants_3_stats_statPerk1
participants_3_stats_statPerk2
participants_3_timeline_participantId
participants_3_timeline_creepsPerMinDeltas_0-10
participants_3_timeline_xpPerMinDeltas_0-10
participants_3_timeline_goldPerMinDeltas_0-10
participants_3_timeline_csDiffPerMinDeltas_0-10
participants_3_timeline_xpDiffPerMinDeltas_0-10
participants_3_timeline_damageTakenPerMinDeltas_0-10
participants_3_timeline_damageTakenDiffPerMinDeltas_0-10
participants_3_timeline_role
participants_3_timeline_lane
participants_4_participantId
participants_4_teamId
participants_4_championId
participants_4_spell1Id
participants_4_spell2Id
participants_4_stats_participantId
participants_4_stats_win
participants_4_stats_item0
participants_4_stats_item1
participants_4_stats_item2
participants_4_stats_item3
participants_4_stats_item4
participants_4_stats_item5
participants_4_stats_item6
participants_4_stats_kills
participants_4_stats_deaths
participants_4_stats_assists
participants_4_stats_largestKillingSpree
participants_4_stats_largestMultiKill
participants_4_stats_killingSprees
participants_4_stats_longestTimeSpentLiving
participants_4_stats_doubleKills
participants_4_stats_tripleKills
participants_4_stats_quadraKills
participants_4_stats_pentaKills
participants_4_stats_unrealKills
participants_4_stats_totalDamageDealt
participants_4_stats_magicDamageDealt
participants_4_stats_physicalDamageDealt
participants_4_stats_trueDamageDealt
participants_4_stats_largestCriticalStrike
participants_4_stats_totalDamageDealtToChampions
participants_4_stats_magicDamageDealtToChampions
participants_4_stats_physicalDamageDealtToChampions
participants_4_stats_trueDamageDealtToChampions
participants_4_stats_totalHeal
participants_4_stats_totalUnitsHealed
participants_4_stats_damageSelfMitigated
participants_4_stats_damageDealtToObjectives
participants_4_stats_damageDealtToTurrets
participants_4_stats_visionScore
participants_4_stats_timeCCingOthers
participants_4_stats_totalDamageTaken
participants_4_stats_magicalDamageTaken
participants_4_stats_physicalDamageTaken
participants_4_stats_trueDamageTaken
participants_4_stats_goldEarned
participants_4_stats_goldSpent
participants_4_stats_turretKills
participants_4_stats_inhibitorKills
participants_4_stats_totalMinionsKilled
participants_4_stats_neutralMinionsKilled
participants_4_stats_totalTimeCrowdControlDealt
participants_4_stats_champLevel
participants_4_stats_visionWardsBoughtInGame
participants_4_stats_sightWardsBoughtInGame
participants_4_stats_firstBloodKill
participants_4_stats_firstBloodAssist
participants_4_stats_firstTowerKill
participants_4_stats_firstTowerAssist
participants_4_stats_firstInhibitorKill
participants_4_stats_firstInhibitorAssist
participants_4_stats_combatPlayerScore
participants_4_stats_objectivePlayerScore
participants_4_stats_totalPlayerScore
participants_4_stats_totalScoreRank
participants_4_stats_playerScore0
participants_4_stats_playerScore1
participants_4_stats_playerScore2
participants_4_stats_playerScore3
participants_4_stats_playerScore4
participants_4_stats_playerScore5
participants_4_stats_playerScore6
participants_4_stats_playerScore7
participants_4_stats_playerScore8
participants_4_stats_playerScore9
participants_4_stats_perk0
participants_4_stats_perk0Var1
participants_4_stats_perk0Var2
participants_4_stats_perk0Var3
participants_4_stats_perk1
participants_4_stats_perk1Var1
participants_4_stats_perk1Var2
participants_4_stats_perk1Var3
participants_4_stats_perk2
participants_4_stats_perk2Var1
participants_4_stats_perk2Var2
participants_4_stats_perk2Var3
participants_4_stats_perk3
participants_4_stats_perk3Var1
participants_4_stats_perk3Var2
participants_4_stats_perk3Var3
participants_4_stats_perk4
participants_4_stats_perk4Var1
participants_4_stats_perk4Var2
participants_4_stats_perk4Var3
participants_4_stats_perk5
participants_4_stats_perk5Var1
participants_4_stats_perk5Var2
participants_4_stats_perk5Var3
participants_4_stats_perkPrimaryStyle
participants_4_stats_perkSubStyle
participants_4_stats_statPerk0
participants_4_stats_statPerk1
participants_4_stats_statPerk2
participants_4_timeline_participantId
participants_4_timeline_creepsPerMinDeltas_0-10
participants_4_timeline_xpPerMinDeltas_0-10
participants_4_timeline_goldPerMinDeltas_0-10
participants_4_timeline_csDiffPerMinDeltas_0-10
participants_4_timeline_xpDiffPerMinDeltas_0-10
participants_4_timeline_damageTakenPerMinDeltas_0-10
participants_4_timeline_damageTakenDiffPerMinDeltas_0-10
participants_4_timeline_role
participants_4_timeline_lane






------------------------------------------------------------------------------------------
gameDuration
teams_teamId
teams_1_win
teams_1_firstBlood
teams_1_firstTower
teams_1_firstInhibitor
teams_1_firstBaron
teams_1_firstDragon
teams_1_firstRiftHerald
teams_1_towerKills
teams_1_inhibitorKills
teams_1_baronKills
teams_1_dragonKills
teams_1_riftHeraldKills













gameId
platformId
gameCreation
gameDuration
queueId
mapId
seasonId
gameVersion
gameMode
gameType
teams_0_teamId
teams_0_win
teams_0_firstBlood
teams_0_firstTower
teams_0_firstInhibitor
teams_0_firstBaron
teams_0_firstDragon
teams_0_firstRiftHerald
teams_0_towerKills
teams_0_inhibitorKills
teams_0_baronKills
teams_0_dragonKills
teams_0_vilemawKills
teams_0_riftHeraldKills
teams_0_dominionVictoryScore
teams_1_teamId
teams_1_win
teams_1_firstBlood
teams_1_firstTower
teams_1_firstInhibitor
teams_1_firstBaron
teams_1_firstDragon
teams_1_firstRiftHerald
teams_1_towerKills
teams_1_inhibitorKills
teams_1_baronKills
teams_1_dragonKills
teams_1_vilemawKills
teams_1_riftHeraldKills
teams_1_dominionVictoryScore
participants_0_participantId
participants_0_teamId
participants_0_championId
participants_0_spell1Id
participants_0_spell2Id
participants_0_stats_participantId
participants_0_stats_win
participants_0_stats_item0
participants_0_stats_item1
participants_0_stats_item2
participants_0_stats_item

In [89]:
pprint(a)

{'gameCreation': 1611228224946,
 'gameDuration': 1927,
 'gameId': 4938438709,
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'gameVersion': '11.2.353.8505',
 'mapId': 11,
 'participantIdentities_0_participantId': 1,
 'participantIdentities_0_player_accountId': 'WstIH4CJ4my6D1vfns0X0wdFBjLhtHeHaIuaziEszpRct0OestbX_0mY',
 'participantIdentities_0_player_currentAccountId': 'WstIH4CJ4my6D1vfns0X0wdFBjLhtHeHaIuaziEszpRct0OestbX_0mY',
 'participantIdentities_0_player_currentPlatformId': 'KR',
 'participantIdentities_0_player_matchHistoryUri': '/v1/stats/player_history/KR/2002081850125888',
 'participantIdentities_0_player_platformId': 'KR',
 'participantIdentities_0_player_profileIcon': 4568,
 'participantIdentities_0_player_summonerId': 'BgdbCThuvxo_koDJ7f16wdVBdYwcnDf8PEhxuNa9kv0GrZI',
 'participantIdentities_0_player_summonerName': 'Siod0me',
 'participantIdentities_1_participantId': 2,
 'participantIdentities_1_player_accountId': 'scO_rjJ4EfhJz-Mrt6hj8v30ti4i5IUvnT6EGXyHDqGTnNRRTt

In [ ]:
new branch